In [1]:
%%time
def ok():
    print('> ok')
ok()

> ok
CPU times: user 642 µs, sys: 82 µs, total: 724 µs
Wall time: 548 µs


In [2]:
%%time



from pyspark.sql import SparkSession
from pyspark import sql
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, TimestampType, IntegerType, DoubleType
from pyspark.sql.functions import unix_timestamp, to_timestamp, col
from pyspark.sql.functions import udf
import re

def ffunc(x, y, z, m):
    fix = "        "
    if m == 1 or m == -1:
        fix = "+" + fix
        entrance_name, entrance_id, station_name = x, y, z
        if x.endswith(" К") or x.endswith("КОЛЬЦЕВАЯ)"):
            fix = fix + " Кольцевая линия"
        elif x.endswith(" Р") or x.endswith(")"):
            fix = fix + " Радиальная линия"
    if m == 2 or m == -2:
        fix = "+" + fix
        station, global_id, line = x, y, z
        if z is None:
            fix = fix
        elif z.endswith("Московское центральное кольцо"):
            fix = fix + " МЦК"
        elif z.endswith("Московская монорельсовая транспортная система"):
            fix = fix + " Монорельс"
        elif z.endswith("Третий пересадочный контур"):
            fix = fix + " ТПК"
        elif z.endswith("Кольцевая линия"):
            fix = fix + " Кольцевая линия"
        else:
            fix = fix + " Радиальная линия"
        if m < 0 and line is not None: fix = fix + ' ' + line
    if m == 3 or m == -3:
        fix = "+" + fix
        station_name_rus, x_line_id, name = x, y, z
        if z is None:
            fix = fix
        elif z.endswith("Московское центральное кольцо"):
            fix = fix + " МЦК"
        elif z.endswith("Московская монорельсовая транспортная система"):
            fix = fix + " Монорельс"
        elif z.endswith("Третий пересадочный контур"):
            fix = fix + " ТПК"
        elif z.endswith("Кольцевая линия"):
            fix = fix + " Кольцевая линия"
        else:
            fix = fix + " Радиальная линия"
        if m < 0 and name is not None: fix = fix + ' ' + name
    return x + "    " + ' '.join(list(fix))

def get_fix_udf():
    return udf(lambda a, b, c, d: ffunc(a, b, c, d), StringType())

def afunc(ignor, txt):
    return '' if txt is None else ' '.join([re.sub(r"_[_]*", "_", re.sub(r"[^a-zа-яё+]", "_",
                                            (txt[i-1] if i - 1 >= 0 else '_').lower() + 
                                            txt[i].lower() +
                                            (txt[i+1] if i + 1 <= len(txt) - 1 else '-').lower()))
                                     for i in range(0, len(txt))]) 

def get_arr_udf():
    return udf(lambda a, b: afunc(a, b), StringType())

def get_key_udf():
    return udf(lambda txt: '' if txt is None or not len(txt)
        else txt[0].lower(), StringType())

def wfunc(a, b):
    x = set(a.split(' '))
    y = set(b.split(' '))
    sz = len(x.intersection(y))
    szx = len(x)
    szy = len(y)
    sign = -1.0 if a.index("+") == -1 or b.index("+") == -1 else 1.0
    return 0.0 if szx + szy == 0 else sign * sz / (szx + szy) 

def get_weight_udf():
    return udf(lambda a, b: wfunc(a, b), DoubleType())

def filter_df(df, col0, col1, col2):
    import pyspark.sql.functions as F
    import pyspark.sql.window as W
    tmp = df.withColumn("_FLT", F.row_number().over(W.Window.partitionBy(col2).orderBy(col0)))
    tmp = tmp.filter(F.col("_FLT") == 1).select(col2.alias("_F2"))
    res = df.join(tmp, col2 == tmp["_F2"], how="inner")
    return res

def ping():
    print("pong")

if __name__ == "__main__":
    import sys
    ping()

    
    
ok()

pong
> ok
CPU times: user 109 ms, sys: 3.92 ms, total: 113 ms
Wall time: 113 ms


In [24]:
%%time
import shutil
import glob
shutil.copyfile("/home/user/ALBINA/DENIS/METRO_MAPPING/entrance_station_codes.csv", "./denis_mapping_e.csv")
shutil.copyfile("/home/user/ALBINA/DENIS/METRO_MAPPING/data_mos_ru_stations.csv", "./denis_mapping_d.csv")
shutil.copyfile("/home/user/ALBINA/DENIS/METRO_MAPPING/maxima_station_codes.csv", "./denis_mapping_m.csv")
shutil.copyfile("/home/user/ALBINA/DENIS/METRO_MAPPING/x_line_codes.csv", "./denis_mapping_x.csv")
#for file in glob.glob(r"./denis_mapping_1to2.csv/part*"): shutil.copyfile(file, "./denis_mapping_1to2fix.csv")
#for file in glob.glob(r"./denis_mapping_2to3.csv/part*"): shutil.copyfile(file, "./denis_mapping_2to3fix.csv")
#for file in glob.glob(r"./denis_mapping_result.csv/part*"): shutil.copyfile(file, "./asana_entrances_metro_maxima.csv")
#for file in glob.glob(r"./denis_mapping_result2.csv/part*"): shutil.copyfile(file, "./asana_metro_maxima.csv")
ok()

> ok
CPU times: user 1.78 ms, sys: 236 µs, total: 2.02 ms
Wall time: 1.87 ms


In [4]:
%%time
import os

os.environ['SPARK_HOME'] = "/usr/local/spark/spark-2.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/user/.sdkman/candidates/java/current"

ok()

> ok
CPU times: user 449 µs, sys: 40 µs, total: 489 µs
Wall time: 500 µs


In [5]:
%%time
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark import sql 
sq = sql.SQLContext(spark.sparkContext)

from pyspark.sql.types import StringType, StructType, StructField, TimestampType, IntegerType
from pyspark.sql.functions import unix_timestamp, to_timestamp, col

def preview(df):
    df.show(n=3, truncate=False)
    #pass ### 

ok()

> ok
CPU times: user 27.9 ms, sys: 708 µs, total: 28.7 ms
Wall time: 2.96 s


In [6]:
%%time
df_e_src = sq.read.csv('./denis_mapping_e.csv',header = True)
df_e_src = sq.createDataFrame(df_e_src.rdd, StructType([StructField("ENTRANCE_ID", StringType()),
                                             StructField("ENTRANCE_NAME", StringType()),
                                             StructField("STATION_NAME", StringType())]))
preview(df_e_src)
ok() ### DF_E_SRC

+-----------+--------------------------------+---------------------+
|ENTRANCE_ID|ENTRANCE_NAME                   |STATION_NAME         |
+-----------+--------------------------------+---------------------+
|20         |Бульвар Рокоссовского (Северный)|Бульвар Рокоссовского|
|21         |Бульвар Рокоссовского (Южный)   |Бульвар Рокоссовского|
|23         |Черкизовская (Северный)         |Черкизовская         |
+-----------+--------------------------------+---------------------+
only showing top 3 rows

> ok
CPU times: user 202 ms, sys: 20.1 ms, total: 222 ms
Wall time: 3.83 s


In [7]:
%%time
df_d_src = sq.read.csv('./denis_mapping_d.csv',header = True)
preview(df_d_src)
ok() ### DF_D_SRC

+-------------+-------------------------+---------------------------------------+---------+-------------------------+---------+---+----+
|STATION      |LINE                     |ADMAREA                                |GLOBAL_ID|DISTRICT                 |STATUS   |ID |NAN |
+-------------+-------------------------+---------------------------------------+---------+-------------------------+---------+---+----+
|Третьяковская|Калининская линия        |Центральный административный округ     |58701962 |район Замоскворечье      |действует|136|null|
|Медведково   |Калужско-Рижская линия   |Северо-Восточный административный округ|58701963 |район Северное Медведково|действует|86 |null|
|Первомайская |Арбатско-Покровская линия|Восточный административный округ       |58701964 |район Измайлово          |действует|41 |null|
+-------------+-------------------------+---------------------------------------+---------+-------------------------+---------+---+----+
only showing top 3 rows

> ok
CPU times: 

In [8]:
%%time
df_m_src = sq.read.csv('./denis_mapping_m.csv',header = True)
preview(df_m_src)
ok() ### DF_M_SRC

+-------+-------+----------+----------------------+--------------------------+
|CITY_ID|LINE_ID|STATION_ID|STATION_NAME_RUS      |STATION_NAME_ENG          |
+-------+-------+----------+----------------------+--------------------------+
|1      |1      |1         |Бульвар Рокоссовского |Bulvar Rokossovskogo      |
|1      |1      |2         |Черкизовская          |Cherkizovskaya            |
|1      |1      |3         |Преображенская площадь|Preobrazhenskaya Ploshchad|
+-------+-------+----------+----------------------+--------------------------+
only showing top 3 rows

> ok
CPU times: user 1.66 ms, sys: 146 µs, total: 1.8 ms
Wall time: 159 ms


In [9]:
%%time
df_x_src = sq.read.csv('./denis_mapping_x.csv',header = True)
df_x_src = df_x_src.withColumn("X_LINE_ID", df_x_src["LINE_ID"]).drop("LINE_ID")
df_x_src = df_x_src.withColumn("LINE_NAME_ENG", df_x_src["COLOR"])
preview(df_x_src)
ok() ### DF_X_SRC

+-------+-------------------------+---------+-------------+
|COLOR  |NAME                     |X_LINE_ID|LINE_NAME_ENG|
+-------+-------------------------+---------+-------------+
|#EF1E25|Сокольническая линия     |1        |#EF1E25      |
|#029A55|Замоскворецкая линия     |2        |#029A55      |
|#0252A2|Арбатско-Покровская линия|3        |#0252A2      |
+-------+-------------------------+---------+-------------+
only showing top 3 rows

> ok
CPU times: user 4.65 ms, sys: 0 ns, total: 4.65 ms
Wall time: 200 ms


In [10]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_e_map = df_e_src
df_e_map = df_e_map.withColumn("_1VAL", F.concat(F.col("ENTRANCE_NAME")))
df_e_map = df_e_map.withColumn("_1KEY", get_key_udf()(F.col("_1VAL")))
df_e_map = df_e_map.withColumn("_1FIX", get_fix_udf()(F.col("_1VAL"), F.col("ENTRANCE_ID"), F.col("STATION_NAME"), F.lit(1)))
df_e_map = df_e_map.withColumn("_1ARR", get_arr_udf()(F.col("_1VAL"), F.col("_1FIX")))
df_e_map = df_e_map.withColumn("_1ROW", F.row_number().over(W.Window.partitionBy(F.lit(1)).orderBy(F.lit(1))))
preview(df_e_map)
ok() ### DF_E_MAP

+-----------+--------------------------------+---------------------+--------------------------------+-----+---------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|ENTRANCE_ID|ENTRANCE_NAME                   |STATION_NAME         |_1VAL                           |_1KEY|_1FIX                                                                                  |_1ARR                                                                                                                                                                                                                                                                                                    |_1ROW|


In [11]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_d_map = df_d_src
df_d_map = df_d_map.withColumn("STATION_METRO", F.concat(F.col("STATION")))
df_d_map = df_d_map.withColumn("LINE_METRO", F.concat(F.col("LINE")))
df_d_map = df_d_map.withColumn("_2VAL", F.concat(F.col("STATION")))
df_d_map = df_d_map.withColumn("_2KEY", get_key_udf()(F.col("_2VAL")))
df_d_map = df_d_map.withColumn("_2FIX", get_fix_udf()(F.col("_2VAL"), F.col("GLOBAL_ID"), F.col("LINE"), F.lit(-2)))
df_d_map = df_d_map.withColumn("_2ARR", get_arr_udf()(F.col("_2VAL"), F.col("_2FIX")))
df_d_map = df_d_map.withColumn("_2ROW", F.row_number().over(W.Window.partitionBy(F.lit(1)).orderBy(F.lit(1))))
preview(df_d_map)
ok() ### DF_D_MAP

+-------------+-------------------------+---------------------------------------+---------+-------------------------+---------+---+----+-------------+-------------------------+-------------+-----+-----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|STATION      |LINE                     |ADMAREA                                |GLOBAL_ID|DISTRICT                 |STATUS   |ID |NAN |STATION_METRO|LINE_METRO               |_2VAL        |_2KEY|_2FIX                                                          

In [12]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_x_map = df_m_src.join(df_x_src, df_m_src.LINE_ID == df_x_src.X_LINE_ID, "left_outer")
df_x_map = df_x_map.withColumn("STATION_MAXIMA", F.concat(F.col("STATION_ID")))
df_x_map = df_x_map.withColumn("_3VAL", F.concat(F.col("STATION_NAME_RUS")))
df_x_map = df_x_map.withColumn("_3KEY", get_key_udf()(F.col("_3VAL")))
df_x_map = df_x_map.withColumn("_3FIX", get_fix_udf()(F.col("_3VAL"), F.col("X_LINE_ID"), F.col("NAME"), F.lit(-3)))
df_x_map = df_x_map.withColumn("_3ARR", get_arr_udf()(F.col("_3VAL"), F.col("_3FIX")))
df_x_map = df_x_map.withColumn("_3ROW", F.row_number().over(W.Window.partitionBy(F.lit(1)).orderBy(F.lit(1))))
preview(df_x_map)
ok() ### DF_X_MAP

+-------+-------+----------+----------------------+--------------------------+-------+--------------------+---------+-------------+--------------+----------------------+-----+-----------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|CITY_ID|LINE_ID|STATION_ID|STATION_NAME_RUS      |STATION_NAME_ENG          |COLOR  |NAME                |X_LINE_ID|LINE_NAME_ENG|STATION_MAXIMA|_3VAL                 |_3KEY|_3FIX                                                                                                

In [13]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_1 = df_e_map.select("_1ROW", "ENTRANCE_ID")
df_1.coalesce(1).write.csv("./denis_mapping_1.csv", header=True, mode="Overwrite")
preview(df_1)
ok() ### DF_1

+-----+-----------+
|_1ROW|ENTRANCE_ID|
+-----+-----------+
|1    |20         |
|2    |21         |
|3    |23         |
+-----+-----------+
only showing top 3 rows

> ok
CPU times: user 9.03 ms, sys: 6.32 ms, total: 15.4 ms
Wall time: 946 ms


In [14]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_2 = df_d_map.select("_2ROW", "STATION_METRO", "LINE_METRO")
df_2.coalesce(1).write.csv("./denis_mapping_2.csv", header=True, mode="Overwrite")
preview(df_2)
ok() ### DF_2

+-----+-------------+-------------------------+
|_2ROW|STATION_METRO|LINE_METRO               |
+-----+-------------+-------------------------+
|1    |Третьяковская|Калининская линия        |
|2    |Медведково   |Калужско-Рижская линия   |
|3    |Первомайская |Арбатско-Покровская линия|
+-----+-------------+-------------------------+
only showing top 3 rows

> ok
CPU times: user 4.77 ms, sys: 0 ns, total: 4.77 ms
Wall time: 816 ms


In [15]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_3 = df_x_map.select("_3ROW", "STATION_MAXIMA")
df_3.coalesce(1).write.csv("./denis_mapping_3.csv", header=True, mode="Overwrite")
preview(df_3)
ok() ### DF_3

+-----+--------------+
|_3ROW|STATION_MAXIMA|
+-----+--------------+
|1    |1             |
|2    |2             |
|3    |3             |
+-----+--------------+
only showing top 3 rows

> ok
CPU times: user 770 µs, sys: 3.65 ms, total: 4.42 ms
Wall time: 711 ms


In [16]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_1to2 = df_e_map.join(df_d_map, df_e_map["_1KEY"] == df_d_map["_2KEY"])
df_1to2 = df_1to2.withColumn("W_1TO2", get_weight_udf()(F.col("_1ARR"), F.col("_2ARR")))
df_1to2 = df_1to2.select("_1ROW", "_2ROW", "ENTRANCE_ID", "_1VAL", "_1KEY", "_2KEY", "_2VAL", "LINE_METRO",
                         "W_1TO2", "_1FIX", "_2FIX")
df_1to2 = df_1to2.sort(F.col("_1ROW"), F.col("W_1TO2").desc())
df_1to2.coalesce(1).write.csv("./denis_mapping_1to2.csv", header=True, mode="Overwrite")
preview(df_1to2)
ok() ### DF_1TO2

+-----+-----+-----------+--------------------------------+-----+-----+------------------------+-------------------------------+------------------+---------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|_1ROW|_2ROW|ENTRANCE_ID|_1VAL                           |_1KEY|_2KEY|_2VAL                   |LINE_METRO                     |W_1TO2            |_1FIX                                                                                  |_2FIX                                                                                                                                          |
+-----+-----+-----------+--------------------------------+-----+-----+------------------------+-------------------------------+------------------+---------------------------------------------------------------------------------------+--------

In [17]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_2to3 = df_d_map.join(df_x_map, df_d_map["_2KEY"] == df_x_map["_3KEY"])
df_2to3 = df_2to3.withColumn("W_2TO3", get_weight_udf()(F.col("_2ARR"), F.col("_3ARR")))
df_2to3 = df_2to3.select("_2ROW", "_3ROW", "LINE_METRO", "_2VAL", "_2KEY", "_3KEY", "_3VAL", "NAME", "STATION_ID",
                         "LINE_ID", "STATION_MAXIMA", "STATION_NAME_ENG", "LINE_NAME_ENG",
                         "ADMAREA", "GLOBAL_ID", "DISTRICT", "STATUS", "W_2TO3", "_2FIX", "_3FIX")
df_2to3 = df_2to3.sort(F.col("_2ROW"), F.col("W_2TO3").desc())
df_2to3.coalesce(1).write.csv("./denis_mapping_2to3.csv", header=True, mode="Overwrite")
preview(df_2to3)
ok() ### DF_2TO3

+-----+-----+-----------------+-------------+-----+-----+-------------+----------------------+----------+-------+--------------+----------------+-------------+----------------------------------+---------+-------------------+---------+-------------------+--------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+
|_2ROW|_3ROW|LINE_METRO       |_2VAL        |_2KEY|_3KEY|_3VAL        |NAME                  |STATION_ID|LINE_ID|STATION_MAXIMA|STATION_NAME_ENG|LINE_NAME_ENG|ADMAREA                           |GLOBAL_ID|DISTRICT           |STATUS   |W_2TO3             |_2FIX                                                                                                   |_3FIX                                                                                                             |
+-----+-----+-----------------+-------------+-----

In [18]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_1to2_max = df_1to2.select("_1ROW","W_1TO2").groupBy(F.col("_1ROW").alias("R")).agg(
    F.max("W_1TO2").alias("W")).filter(F.col("W") >= 0)
df_1to2_tmp = df_1to2.join(df_1to2_max, [df_1to2_max["R"] == df_1to2["_1ROW"],
                                        df_1to2_max["W"] == df_1to2["W_1TO2"]], how="inner").sort("R").drop("R").drop("W")
df_1to2_tmp.coalesce(1).write.csv("./denis_mapping_1to2tmp.csv", header=True, mode="Overwrite")
preview(df_1to2_tmp)

df_1to2_fix = sq.read.csv('./denis_mapping_1to2fix.csv',header = True)
preview(df_1to2_fix)
ok() ### DF_1TO2_FIX

+-----+-----+-----------+--------------------------------+-----+-----+---------------------+--------------------+------------------+---------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|_1ROW|_2ROW|ENTRANCE_ID|_1VAL                           |_1KEY|_2KEY|_2VAL                |LINE_METRO          |W_1TO2            |_1FIX                                                                                  |_2FIX                                                                                                                 |
+-----+-----+-----------+--------------------------------+-----+-----+---------------------+--------------------+------------------+---------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------

In [19]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_2to3_max = df_2to3.select("_2ROW", "W_2TO3").groupBy(F.col("_2ROW").alias("R")).agg(
    F.max("W_2TO3").alias("W")).filter(F.col("W") >= 0)
df_2to3_tmp = df_2to3.join(df_2to3_max, [df_2to3_max["R"] == df_2to3["_2ROW"],
                                        df_2to3_max["W"] == df_2to3["W_2TO3"]], how="inner").sort("R").drop("R").drop("W")
df_2to3_tmp.coalesce(1).write.csv("./denis_mapping_2to3tmp.csv", header=True, mode="Overwrite")
preview(df_2to3_tmp)

df_2to3_fix = sq.read.csv('./denis_mapping_2to3fix.csv',header = True)
preview(df_2to3_fix)
ok() ### DF_2TO3_FIX

+-----+-----+-------------------------+-------------+-----+-----+-------------+-------------------------+----------+-------+--------------+----------------+-------------+---------------------------------------+---------+-------------------------+---------+------+-----------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|_2ROW|_3ROW|LINE_METRO               |_2VAL        |_2KEY|_3KEY|_3VAL        |NAME                     |STATION_ID|LINE_ID|STATION_MAXIMA|STATION_NAME_ENG|LINE_NAME_ENG|ADMAREA                                |GLOBAL_ID|DISTRICT                 |STATUS   |W_2TO3|_2FIX                                                                                                                  |_3FIX                                                                                                            

In [20]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_1to2_fix = filter_df(df_1to2_tmp, F.col("W_1TO2"), F.col("_1ROW"), F.col("_2ROW"))
df_2to3_fix = filter_df(df_2to3_tmp, F.col("W_2TO3"), F.col("_2ROW"), F.col("_3ROW"))
df_proto = df_e_map.select(F.col("_1VAL").alias("C0"), F.col("ENTRANCE_ID").alias("C1"), F.col("_1ROW"))
df_proto = df_proto.join(df_1to2_fix, df_proto["_1ROW"] == df_1to2_fix["_1ROW"], how="left_outer")
df_proto = df_proto.select(F.col("C0"), F.col("C1"), F.col("_2ROW"), F.col("_2VAL").alias("C2"), F.col("LINE_METRO").alias("C3"))
df_proto = df_proto.join(df_2to3_fix, df_proto["_2ROW"] == df_2to3_fix["_2ROW"], how="left_outer")
df_proto = df_proto.select(F.col("C0"), F.col("C1"), F.col("C2"), F.col("C3"),
                           F.col("STATION_MAXIMA").alias("C4"), F.col("_3VAL").alias("C5"), F.col("NAME").alias("C6"))
preview(df_proto)
ok() ### DF_PROTO

+--------------------------------+---+---------------------+-------------------------------+---+---------------------+-------------------------------+
|C0                              |C1 |C2                   |C3                             |C4 |C5                   |C6                             |
+--------------------------------+---+---------------------+-------------------------------+---+---------------------+-------------------------------+
|Цветной Бульвар                 |370|Цветной бульвар      |Серпуховско-Тимирязевская линия|153|Цветной бульвар      |Серпуховско-Тимирязевская линия|
|Бульвар Рокоссовского (Северный)|20 |Бульвар Рокоссовского|Сокольническая линия           |1  |Бульвар Рокоссовского|Сокольническая линия           |
|Бульвар Рокоссовского (Южный)   |21 |Бульвар Рокоссовского|Сокольническая линия           |1  |Бульвар Рокоссовского|Сокольническая линия           |
+--------------------------------+---+---------------------+-------------------------------+--

In [21]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T
df_result = df_proto.withColumn("ENTRANCE_NAME",
                                 F.when(F.col("C0").isNull(), F.lit("(null)")).otherwise(F.col("C0"))).drop("C0")
df_result = df_result.withColumn("ENTRANCE_ID",
                                 F.when(F.col("C1").isNull(),
                                        F.lit(0)).otherwise(F.col("C1").cast(T.IntegerType()))).drop("C1")
df_result = df_result.withColumn("STATION_METRO",
                                 F.when(F.col("C2").isNull(), F.lit("(null)")).otherwise(F.col("C2"))).drop("C2")
df_result = df_result.withColumn("LINE_METRO",
                                 F.when(F.col("C3").isNull(), F.lit("(null)")).otherwise(F.col("C3"))).drop("C3")
df_result = df_result.withColumn("STATION_ID_MAXIMA",
                                 F.when(F.col("C4").isNull(), F.lit("(null)")).otherwise(F.col("C4"))).drop("C4")
df_result = df_result.withColumn("STATION_MAXIMA",
                                 F.when(F.col("C5").isNull(), F.lit("(null)")).otherwise(F.col("C5"))).drop("C5")
df_result = df_result.withColumn("LINE_MAXIMA",
                                 F.when(F.col("C6").isNull(), F.lit("(null)")).otherwise(F.col("C6"))).drop("C6")
df_result = df_result.sort(F.col("ENTRANCE_ID"))
df_result.coalesce(1).write.csv("./denis_mapping_result.csv", header=True, mode="Overwrite")
preview(df_result)
ok() ### DF_RESULT

+--------------------------------+-----------+---------------------+--------------------+-----------------+---------------------+--------------------+
|ENTRANCE_NAME                   |ENTRANCE_ID|STATION_METRO        |LINE_METRO          |STATION_ID_MAXIMA|STATION_MAXIMA       |LINE_MAXIMA         |
+--------------------------------+-----------+---------------------+--------------------+-----------------+---------------------+--------------------+
|Бульвар Рокоссовского (Северный)|20         |Бульвар Рокоссовского|Сокольническая линия|1                |Бульвар Рокоссовского|Сокольническая линия|
|Бульвар Рокоссовского (Южный)   |21         |Бульвар Рокоссовского|Сокольническая линия|1                |Бульвар Рокоссовского|Сокольническая линия|
|Черкизовская (Северный)         |23         |Черкизовская         |Сокольническая линия|2                |Черкизовская         |Сокольническая линия|
+--------------------------------+-----------+---------------------+--------------------+-----

In [22]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
df_proto2 = df_d_map.select(F.col("_2VAL").alias("C0"), F.col("LINE_METRO").alias("C1"), F.col("_2ROW"))
df_proto2 = df_proto2.join(df_2to3_fix, df_proto2["_2ROW"] == df_2to3_fix["_2ROW"], how="left_outer")
df_proto2 = df_proto2.select(F.col("C0"), F.col("C1"),
                             F.col("LINE_ID").alias("C2"), F.col("STATION_MAXIMA").alias("C3"),
                             F.col("STATION_NAME_ENG").alias("C4"), F.col("LINE_NAME_ENG").alias("C5"),
                             F.col("ADMAREA").alias("C6"), F.col("GLOBAL_ID").alias("C7"),
                             F.col("DISTRICT").alias("C8"), F.col("STATUS").alias("C9"),
                             F.col("STATION_ID").alias("C10"))
preview(df_proto2)
ok() ### DF_PROTO2

+---------------------+-------------------------------+---+---+--------------------+-------+----------------------------------+--------+-----------------+---------+---+
|C0                   |C1                             |C2 |C3 |C4                  |C5     |C6                                |C7      |C8               |C9       |C10|
+---------------------+-------------------------------+---+---+--------------------+-------+----------------------------------+--------+-----------------+---------+---+
|Цветной бульвар      |Серпуховско-Тимирязевская линия|9  |153|Tsvetnoy Bulvar     |#ACADAF|Центральный административный округ|58702110|Тверской район   |действует|153|
|Бульвар Рокоссовского|Сокольническая линия           |1  |1  |Bulvar Rokossovskogo|#EF1E25|Восточный административный округ  |58702205|район Богородское|действует|1  |
|Автозаводская        |Замоскворецкая линия           |2  |32 |Avtozavodskaya      |#029A55|Южный административный округ      |58701993|Даниловский район|д

In [23]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T
df_result2 = df_proto2.withColumn("STATION_NAME",
                                 F.when(F.col("C0").isNull(), F.lit("(null)")).otherwise(F.col("C0"))).drop("C0")
df_result2 = df_result2.withColumn("LINE_NAME",
                                 F.when(F.col("C1").isNull(), F.lit("(null)")).otherwise(F.col("C1"))).drop("C1")
df_result2 = df_result2.withColumn("LINE_ID",
                                 F.when(F.col("C2").isNull(),
                                        F.lit(0)).otherwise(F.col("C2").cast(T.IntegerType()))).drop("C2")
df_result2 = df_result2.withColumn("STATION_ID_MAXIMA",
                                 F.when(F.col("C3").isNull(), F.lit("(null)")).otherwise(F.col("C3"))).drop("C3")
df_result2 = df_result2.withColumn("STATION_NAME_ENG",
                                 F.when(F.col("C4").isNull(), F.lit("(null)")).otherwise(F.col("C4"))).drop("C4")
df_result2 = df_result2.withColumn("LINE_NAME_ENG",
                                 F.when(F.col("C5").isNull(), F.lit("(null)")).otherwise(F.col("C5"))).drop("C5")
df_result2 = df_result2.withColumn("ADM_AREA",
                                 F.when(F.col("C6").isNull(), F.lit("(null)")).otherwise(F.col("C6"))).drop("C6")
df_result2 = df_result2.withColumn("GLOBAL_ID",
                                 F.when(F.col("C7").isNull(), F.lit("(null)")).otherwise(F.col("C7"))).drop("C7")
df_result2 = df_result2.withColumn("DISTRICT",
                                 F.when(F.col("C8").isNull(), F.lit("(null)")).otherwise(F.col("C8"))).drop("C8")
df_result2 = df_result2.withColumn("STATUS",
                                 F.when(F.col("C9").isNull(), F.lit("(null)")).otherwise(F.col("C9"))).drop("C9")
df_result2 = df_result2.withColumn("STATION_ID_MOSMETRO",
                                 F.when(F.col("C10").isNull(),
                                        F.lit(0)).otherwise(F.col("C10").cast(T.IntegerType()))).drop("C10")
df_result2 = df_result2.sort(F.col("LINE_ID").desc(), F.col("STATION_ID_MOSMETRO"))
df_result2.coalesce(1).write.csv("./denis_mapping_result2.csv", header=True, mode="Overwrite")
preview(df_result2)
ok() ### DF_RESULT2

+---------------+-----------------------------+-------+-----------------+----------------+-------------+-----------------------------------+---------+----------------------+---------+-------------------+
|STATION_NAME   |LINE_NAME                    |LINE_ID|STATION_ID_MAXIMA|STATION_NAME_ENG|LINE_NAME_ENG|ADM_AREA                           |GLOBAL_ID|DISTRICT              |STATUS   |STATION_ID_MOSMETRO|
+---------------+-----------------------------+-------+-----------------+----------------+-------------+-----------------------------------+---------+----------------------+---------+-------------------+
|Битцевский парк|Бутовская линия Лёгкого метро|12     |189              |Bittsevsky Park |#85D4F3      |Юго-Западный административный округ|58702054 |район Ясенево         |действует|189                |
|Лесопарковая   |Бутовская линия Лёгкого метро|12     |190              |Lesoparkovaya   |#85D4F3      |Юго-Западный административный округ|58702092 |район Северное Бутово |действует|1